In [23]:
import pandas as pd
import zipfile
import os
import sys
import numpy as np

train_file = 'train.csv'
train_df = pd.read_csv(train_file)
test_file = 'test.csv'
test_df = pd.read_csv(test_file)
query1 = pd.read_csv('query1.csv')
query2 = pd.read_csv('query2.csv')

test_df.head()


,mutant
0,V1D
1,V1Y
2,V1C
3,V1A
4,V1E


In [17]:
# query1 and query 2 method
test_df['pos'] = test_df['mutant'].str[1:-1].astype(int)
test_df['wt'] = test_df['mutant'].str[0]
test_df['mut'] = test_df['mutant'].str[-1]

query_mutants = test_df.groupby('pos').apply(lambda x: x.sample(n=1, random_state=42))
query_mutants = query_mutants['mutant'].tolist()
print(len(query_mutants))

596


/var/folders/z2/_0c1fw_d62gfdmm0hfw2lfpr0000gn/T/ipykernel_55540/3233668334.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  query_mutants = test_df.groupby('pos').apply(lambda x: x.sample(n=1, random_state=42))


In [28]:
queried

,mutant,DMS_score,sequence
0,V1DN556H,1.397420,MDNEARGNSSLNPCLEGSASSGSESSKDSSRCSTPGLDPERHERLR...
1,N2RA557Q,0.952052,MVREARGNSSLNPCLEGSASSGSESSKDSSRCSTPGLDPERHERLR...
2,E3KV558Y,1.487349,MVNKARGNSSLNPCLEGSASSGSESSKDSSRCSTPGLDPERHERLR...
3,A4MT559I,1.709228,MVNEMRGNSSLNPCLEGSASSGSESSKDSSRCSTPGLDPERHERLR...
4,R5WW560A,1.386366,MVNEAWGNSSLNPCLEGSASSGSESSKDSSRCSTPGLDPERHERLR...
...,...,...,...
95,P100KE651A,1.825341,MVNEARGNSSLNPCLEGSASSGSESSKDSSRCSTPGLDPERHERLR...
96,G101AT652D,1.366558,MVNEARGNSSLNPCLEGSASSGSESSKDSSRCSTPGLDPERHERLR...
97,S102ME653W,1.293076,MVNEARGNSSLNPCLEGSASSGSESSKDSSRCSTPGLDPERHERLR...
98,D103CA654E,1.720167,MVNEARGNSSLNPCLEGSASSGSESSKDSSRCSTPGLDPERHERLR...


In [30]:
queried = pd.concat([query1, query2], ignore_index=True)
queried['pos'] = queried['mutant'].str.extract(r'(\d+)').astype(int)
queried['wt'] = queried['mutant'].str[0]
queried['mut'] = queried['mutant'].str[-1]
pos_counts = queried['pos'].value_counts()
pos_counts


pos
1      1
593    1
583    1
584    1
585    1
      ..
71     1
72     1
73     1
74     1
655    1
Name: count, Length: 200, dtype: int64

In [25]:
train_df['pos'] = train_df['mutant'].str[1:-1].astype(int)
train_df['wt'] = train_df['mutant'].str[0]
train_df['mut'] = train_df['mutant'].str[-1]
pos_counts = train_df['pos'].value_counts()
max(pos_counts)
pos_counts


pos
0      19
48     19
254    19
255    19
258    19
276    19
289    19
290    19
292    19
295    19
296    19
297    19
298    19
302    19
304    19
305    19
308    19
309    19
311    19
314    19
315    19
316    19
318    19
319    19
320    19
331    19
334    19
335    19
336    19
253    19
252    19
251    19
185    19
87     19
88     19
89     19
112    19
126    19
141    19
148    19
151    19
152    19
157    19
158    19
182    19
189    19
250    19
191    19
193    19
195    19
196    19
216    19
220    19
221    19
224    19
227    19
228    19
242    19
247    19
347    19
Name: count, dtype: int64

In [18]:
# query 3 method

# option 1. select 100 random mutants
# remove previously queried mutants

query_mutants = [x for x in query_mutants if x not in query1['mutant'].tolist()]
query_mutants = [x for x in query_mutants if x not in query2['mutant'].tolist()]
print(len(query_mutants))
np.random.seed(42)
query_mutants = np.random.choice(query_mutants, 100, replace=False)
'''

# option 2. find the most underrepresented positions and choose a random mutant from each
# get underrepresented positions from training data
pos_counts = test_df['pos'].value_counts()
pos_counts = pos_counts[pos_counts < 10]
# remove previously queried mutants from test_df
test_df_filtered = test_df[~test_df['mutant'].isin(query1['mutant'].tolist())]
test_df_filtered = test_df_filtered[~test_df_filtered['mutant'].isin(query2['mutant'].tolist())]
# get the mutants that are at underrepresented positions
query_mutants = test_df_filtered[test_df_filtered['pos'].isin(pos_counts.index)]
# choose a random mutant from each position
query_mutants = query_mutants.groupby('pos').apply(lambda x: x.sample(n=1, random_state=42))
query_mutants = query_mutants['mutant'].tolist()
len(query_mutants)
# randomly choose 100 mutants
np.random.seed(42)
query_mutants = np.random.choice(query_mutants, 100, replace=False)
'''
print(len(query_mutants))

396
100


In [4]:
with open('query2.txt', 'w') as f:
    for mutant in query_mutants:
        f.write(mutant + '\n')

In [8]:
train_df['pos'] = train_df['mutant'].str[1:-1].astype(int)
train_df['wt'] = train_df['mutant'].str[0]
train_df['mut'] = train_df['mutant'].str[-1]

pos_var = train_df.groupby('pos')['DMS_score'].var()
high_var_pos = pos_var.nlargest(50).index
mutant_count = train_df['pos'].value_counts()
under_rep_pos = mutant_count.nsmallest(50).index

query_cand = train_df[train_df['pos'].isin(high_var_pos.union(under_rep_pos))]
query_mutants = query_cand.sample(n=100, random_state=42)['mutant'].tolist()

In [11]:
print(query_mutants[:10])
print(len(query_mutants))

['I191N', 'R334N', 'A221N', 'P250M', 'P315Y', 'F319P', 'A112R', 'F189L', 'P276E', 'A112S']
100


In [12]:
with open('query.txt', 'w') as f:
    for mutant in query_mutants:
        f.write(mutant + '\n')